# Fase Beta de la entrega del reto
### ***Secuencia***
1. Generar frecuencias de ventas y simular una venta de 105 clientes.
2. Tomar los volúmenes de las compras de los clientes para determinar la cantidad de camiones necesarios.
3. Sabiendo los camiones, utilizar k-medoids para hacer los mini-tcps.
4. Resolver los mtcps.

### ***Librerías***

In [32]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import math

from scipy.stats import poisson
from scipy.stats import poisson
from scipy.stats import chisquare


from ortools.linear_solver import pywraplp

from scipy.spatial.distance import cdist

import time

## Paso 1. Simular un pedido de 100 clientes

In [33]:
compras = pd.read_excel('informacion_compra.xlsx')

In [34]:
compras.head(3)

,Producto,Unidades,Factura
0,48443,1,799186
1,42877,1,717106
2,48296,1,468125


In [35]:
facturas = []
productos = []
for _,i in compras.iterrows():
    facturas += [i[2]]*i[1]
for _,i in compras.iterrows():
    productos += [i[0]]*i[1]

compras = pd.DataFrame({
    "Factura": facturas,
    "Producto": productos
})

In [36]:
df = compras.groupby(['Factura']).size().reset_index(name='Frequency').Frequency.value_counts()
df = df.reset_index()
df.columns = ['Pedidos', 'Frecuencia']
df['Frecuencia ln'] = np.log(df['Frecuencia'])
valor_medio = np.sum(df['Pedidos'] * df['Frecuencia ln']) / np.sum(df['Frecuencia ln'])
df['Poisson'] = poisson.pmf(df['Pedidos'], valor_medio)
df['valores_esperados'] = df['Poisson'] * np.sum(df['Frecuencia ln'])
df.head(3)

,Pedidos,Frecuencia,Frecuencia ln,Poisson,valores_esperados
0,1,19235,9.864487,0.011672,0.783311
1,2,3546,8.173575,0.036706,2.463277
2,3,1024,6.931472,0.076953,5.164178


In [37]:
df2 = compras.groupby(['Producto']).size().reset_index(name='Frequency').Frequency.value_counts()
df2 = df2.reset_index()
df2.columns = ['Producto', 'Frecuencia']
df2['Frecuencia ln'] = np.log(df2['Frecuencia'])
valor_medio = np.sum(df2['Producto'] * df2['Frecuencia ln']) / np.sum(df2['Frecuencia ln'])
df2['Poisson'] = poisson.pmf(df2['Producto'], valor_medio)
df2['valores_esperados'] = df2['Poisson'] * np.sum(df2['Frecuencia ln'])
df2.head(3)

,Producto,Frecuencia,Frecuencia ln,Poisson,valores_esperados
0,1,1793,7.491645,1.565792e-11,2.886239e-09
1,2,779,6.658011,2.209339e-10,4.072494e-08
2,3,414,6.025866,2.078257e-09,3.830870e-07


In [38]:
def montecarlo(df, n):  
    resultados = []
    for _ in range(n):
        num = np.random.rand()
        for j in range(len(df)):
            if df[j][3] <= num < df[j][4]:
                resultados.append(j)
            
    return resultados

In [39]:
pedidos = df[['Pedidos','Poisson']]
pedidos['Acumulado'] = pedidos.Poisson.cumsum()
pedidos['Inferior'] = [0] + pedidos['Acumulado'].tolist()[:-1]
pedidos['Superior'] = pedidos['Acumulado']
pedidos = pedidos.to_numpy()

C:\Users\jmanu\AppData\Local\Temp\ipykernel_2596\3932492861.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pedidos['Acumulado'] = pedidos.Poisson.cumsum()
C:\Users\jmanu\AppData\Local\Temp\ipykernel_2596\3932492861.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pedidos['Inferior'] = [0] + pedidos['Acumulado'].tolist()[:-1]


In [40]:
#num_pedidos = montecarlo(pedidos,df.Frecuencia.sum())
num_pedidos = montecarlo(pedidos,compras.size)


In [41]:
producto = df2[['Producto','Poisson']]
producto['Acumulado'] = producto.Poisson.cumsum()
producto['Inferior'] = [0] + producto['Acumulado'].tolist()[:-1]
producto['Superior'] = producto['Acumulado']
producto = producto.to_numpy()

C:\Users\jmanu\AppData\Local\Temp\ipykernel_2596\3891866760.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  producto['Acumulado'] = producto.Poisson.cumsum()
C:\Users\jmanu\AppData\Local\Temp\ipykernel_2596\3891866760.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  producto['Inferior'] = [0] + producto['Acumulado'].tolist()[:-1]


In [42]:
envios = [np.array(montecarlo(producto, num)) for num in montecarlo(pedidos,110) if num!=0]
envios_np = envios[:106]

In [43]:
enviosArreglo = np.array([[-1,-1,-1]])
for i in range(len(envios)):
    a = np.unique(np.array(envios[i]), return_counts=True)
    enviosArreglo = np.concatenate((enviosArreglo, np.array([np.array([i]*len(a[0])),a[0],a[1]]).T))
enviosArreglo

array([[ -1,  -1,  -1],
       [  0,  24,   1],
       [  0,  25,   1],
       ...,
       [106,  40,   1],
       [107,  25,   2],
       [107,  32,   1]], dtype=int64)

In [44]:
dp = pd.read_csv('info_productos.csv')
dp.columns = ['Producto', 'Volumen']
nuevo_registro = {"Producto": 0, "Volumen": 0}
dp = pd.concat([pd.DataFrame([nuevo_registro]), dp], ignore_index=True)
VOLU = dp.to_numpy()[:,1]

In [45]:
dimensiones = []
for pedido in envios_np:
    acumulado = 0
    for i in pedido:
        acumulado += VOLU[i]
    dimensiones.append(round(acumulado,4))

In [46]:
from ortools.linear_solver import pywraplp


def create_data_model():
    """Create the data for the example."""
    data = {}
    weights = dimensiones
    data["weights"] = weights
    data["items"] = list(range(len(weights)))
    data["bins"] = data["items"]
    data["bin_capacity"] = 27 #isuzu aprox
    return data



def main():
    data = create_data_model()

    # Create the mip solver with the SCIP backend.
    solver = pywraplp.Solver.CreateSolver("SCIP")

    if not solver:
        return

    # Variables
    # x[i, j] = 1 if item i is packed in bin j.
    x = {}
    for i in data["items"]:
        for j in data["bins"]:
            x[(i, j)] = solver.IntVar(0, 1, "x_%i_%i" % (i, j))

    # y[j] = 1 if bin j is used.
    y = {}
    for j in data["bins"]:
        y[j] = solver.IntVar(0, 1, "y[%i]" % j)

    # Constraints
    # Each item must be in exactly one bin.
    for i in data["items"]:
        solver.Add(sum(x[i, j] for j in data["bins"]) == 1)

    # The amount packed in each bin cannot exceed its capacity.
    for j in data["bins"]:
        solver.Add(
            sum(x[(i, j)] * data["weights"][i] for i in data["items"])
            <= y[j] * data["bin_capacity"]
        )

    # Objective: minimize the number of bins used.
    solver.Minimize(solver.Sum([y[j] for j in data["bins"]]))

    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        num_bins = 0
        for j in data["bins"]:
            if y[j].solution_value() == 1:
                bin_items = []
                bin_weight = 0
                for i in data["items"]:
                    if x[i, j].solution_value() > 0:
                        bin_items.append(i)
                        bin_weight += data["weights"][i]
                if bin_items:
                    num_bins += 1
                    print("Camión # ", j+1)
                    print("Cantidad de clientes # ", len(bin_items))
                    print("  Clientes:", bin_items)
                    print(f"  Volumen total (m^3): {round(bin_weight, 2)}")
                    print()
        print()
        print("Cantidad de camiones:", num_bins)
        print("Capacidad de los camiones:", data["bin_capacity"], "m^3")
        print("Tiempo = ", solver.WallTime(), " milisegundos")
    else:
        print("No existe solución óptima.")


if __name__ == "__main__":
    main()
 

Camión #  1
Cantidad de clientes #  41
  Clientes: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 34, 36, 37, 38, 41, 42, 69, 101]
  Volumen total (m^3): 26.99

Camión #  2
Cantidad de clientes #  31
  Clientes: [33, 35, 39, 40, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 74, 76]
  Volumen total (m^3): 26.98

Camión #  3
Cantidad de clientes #  22
  Clientes: [68, 70, 71, 72, 73, 75, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 96]
  Volumen total (m^3): 26.95

Camión #  4
Cantidad de clientes #  12
  Clientes: [92, 93, 94, 95, 97, 98, 99, 100, 102, 103, 104, 105]
  Volumen total (m^3): 13.12


Cantidad de camiones: 4
Capacidad de los camiones: 27 m^3
Tiempo =  532  milisegundos


## Paso 3. Sabiendo los camiones necesarios, hacer clusters

In [47]:
path = "https://raw.githubusercontent.com/gerardoramc/AlgoritmoDatos/Gerardo/distancias_tiempos2.csv"
df = pd.read_csv(path)
df.head(3)

,Distance,Duration
0,0.00,0:00
1,15.26,0:22
2,14.93,0:17


In [48]:
df['Duration'] = pd.to_datetime(df['Duration'])
df['Duration'] = df['Duration'].dt.strftime('%H:%M')
df['Duration']=df['Duration'].astype('string')

C:\Users\jmanu\AppData\Local\Temp\ipykernel_2596\2034775426.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Duration'] = pd.to_datetime(df['Duration'])


In [49]:

def convert_to_seconds(time_str):
    hours, minutes = map(int, time_str.split(':'))
    total_seconds = hours * 3600 + minutes * 60
    return total_seconds

# Apply the conversion function to the DataFrame column
df['Segundos'] = df['Duration'].apply(convert_to_seconds)

In [50]:
rangos = np.arange(0,11236,106)

In [51]:
cuadrada = []
for i in range(len(rangos)-1):
    cuadrada.append(list((df['Segundos'][rangos[i]:rangos[i+1]])))
result_array = np.append(np.array(cuadrada), [(df['Segundos'][rangos[-1]:])], axis = 0)
result_array.shape

(106, 106)

In [52]:
def remove_first_row_and_column(input_array):
    if input_array.shape[0] <= 1 or input_array.shape[1] <= 1:
        raise ValueError("Input array must have at least 2 rows and 2 columns.")

    new_array = input_array[1:, 1:]
    return new_array


In [53]:
labels = [0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 2, 1, 0, 1, 0, 1, 1, 1,
       1, 2, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1,
       1, 1, 0, 0, 1, 0, 1, 2, 1, 0, 1, 1, 1, 1, 1, 0, 2, 0, 0, 1, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 1, 1, 1, 1]

In [68]:
clusters = {}

In [69]:
my_list = np.array(labels)
for i in range(0,3): 
    indices = np.where(my_list == i)[0]
    #indices = np.add(1, indices)
    clusters.update({i:indices})

In [70]:
clusters

{0: array([  0,   2,   4,   5,  10,  11,  13,  16,  18,  25,  27,  28,  29,
         30,  31,  35,  36,  39,  40,  42,  50,  54,  56,  57,  60,  61,
         64,  68,  69,  71,  75,  81,  83,  84,  87,  88,  90,  91,  92,
         93,  94,  96,  97,  98,  99, 101], dtype=int64),
 1: array([  1,   3,   6,   7,   8,   9,  12,  15,  17,  19,  20,  21,  22,
         24,  26,  32,  33,  34,  37,  38,  41,  43,  44,  45,  46,  47,
         48,  49,  51,  52,  53,  55,  58,  59,  62,  63,  65,  66,  67,
         70,  72,  74,  76,  77,  78,  79,  80,  85,  86,  89,  95, 102,
        103, 104, 105], dtype=int64),
 2: array([ 14,  23,  73,  82, 100], dtype=int64)}

In [57]:
vol_por_clus = {}

for clave, indices in clusters.items():
    pesos = [dimensiones[indice] for indice in indices]
    vol_por_clus[clave] = pesos

vol_por_clus

{0: [2.8959,
  0.6896,
  0.1834,
  0.2005,
  3.4283,
  0.0275,
  1.6,
  0.3827,
  0.0483,
  0.0626,
  0.9853,
  0.1935,
  1.6395,
  0.958,
  1.0486,
  1.908,
  0.0148,
  1.908,
  0.4259,
  0.2896,
  0.307,
  0.5322,
  0.1163,
  0.1799,
  0.1853,
  0.055,
  0.2192,
  1.9217,
  0.0136,
  1.2347,
  0.1883,
  1.0428,
  0.2875,
  1.3045,
  1.5391,
  1.2662,
  0.2148,
  1.7082,
  0.6777,
  1.026,
  0.3446,
  0.1461,
  3.5183,
  1.2079,
  2.8844,
  0.0191],
 1: [0.307,
  1.8341,
  0.9222,
  0.3153,
  1.6085,
  0.0361,
  0.0029,
  0.0661,
  0.0162,
  1.9236,
  0.0802,
  1.3223,
  0.2311,
  1.0113,
  0.3433,
  1.0103,
  1.9812,
  0.0898,
  0.5456,
  0.5921,
  0.0137,
  1.6107,
  0.1638,
  0.156,
  0.176,
  0.1645,
  0.922,
  0.9158,
  0.4137,
  0.9178,
  0.4234,
  2.3099,
  0.8956,
  1.8545,
  1.2248,
  1.6009,
  0.2621,
  4.35,
  0.3266,
  3.6713,
  1.6178,
  0.4481,
  0.0293,
  1.6426,
  0.3584,
  2.907,
  1.9983,
  0.0939,
  1.2324,
  1.057,
  1.3342,
  0.9907,
  0.4498,
  0.1037,
  0.183],


In [58]:
multiplier = 1
trucks_used = 0
start_time=time.time()
chamions = []
for clave, volumenes in vol_por_clus.items():

    print(f"**********CLUSTER # {clave + 1 }**********\n")
    incomplete = True
    while incomplete:
        data = {}
        data["weights"] = volumenes
        #data["values"] = len(data["weights"]) * [1]
        data["values"] =  volumenes
        sum_of_all_weights = sum(data["weights"])

        assert len(data["weights"]) == len(data["values"])
        data["num_items"] = len(data["weights"])
        data["all_items"] = range(data["num_items"])

        data["bin_capacities"] = [27] * multiplier
        data["num_bins"] = len(data["bin_capacities"])
        data["all_bins"] = range(data["num_bins"])

        solver = pywraplp.Solver.CreateSolver("SCIP")

        # x[i, b] = 1 if item i is packed in bin b.
        x = {}
        for i in data["all_items"]:
            for b in data["all_bins"]:
                x[i, b] = solver.BoolVar(f"x_{i}_{b}")

        # Each item is assigned to at most one bin.
        for i in data["all_items"]:
            solver.Add(sum(x[i, b] for b in data["all_bins"]) <= 1)

        # The amount packed in each bin cannot exceed its capacity.
        for b in data["all_bins"]:
            solver.Add(
                sum(x[i, b] * data["weights"][i] for i in data["all_items"])
                <= data["bin_capacities"][b])
        
        # Each bin must contain at least 2 items
        for b in data["all_bins"]:
            solver.Add(sum(x[i, b] for i in data["all_items"]) <= 20)
            
         # Each bin must contain at most 20 items
        '''for b in data["all_bins"]:
            solver.Add(sum(x[i, b] for i in data["all_items"]) <= 20)'''
        
        # Maximize total value of packed items.
        objective = solver.Objective()
        for i in data["all_items"]:
            for b in data["all_bins"]:
                objective.SetCoefficient(x[i, b], data["values"][i])
        objective.SetMaximization()

        start_time=time.time()
        status = solver.Solve()

        if status == pywraplp.Solver.OPTIMAL:

            unused_items = [i for i in data["all_items"] if all(x[i, b].solution_value() == 0 for b in data["all_bins"])]


            used_bins = [b for b in data["all_bins"] if any(x[i, b].solution_value() > 0 for i in data["all_items"])]
            if len(unused_items) != 0:
                multiplier += 1

            if len(unused_items) == 0:

                print(f"Número de camiones requeridos: {len(used_bins)}\n")

                print(f"Valor total empaquetado: {round(objective.Value(), 2)}\n")

                max_value_bin = None
                max_value = 0
                total_weight = 0
                total_value = 0
                for b in used_bins:
                    print(f"*** CAMIÓN # {b+1} ***")
                    print(f'Capacidad: {data["bin_capacities"][b]}')
                    bin_weight = 0
                    bin_value = 0
                    packed_items = []
                    for i in data["all_items"]:
                        if x[i, b].solution_value() > 0:
                            packed_items.append(str(clusters[clave][i]))
                            bin_weight += data["weights"][i]
                            bin_value += data["values"][i]
                    print(f"Clientes empacados: {', '.join(packed_items)}")
                    print(f"Volumen empaquetado en el camión: {round(bin_weight, 4)}\n")
                    #print(f"Valor empaquetado del camión: {bin_value}\n")
                    total_value += bin_value
                    total_weight += bin_weight
                    if bin_value > max_value:
                        max_value = bin_value
                        max_value_bin = b
                    chamions.append([int(x) for x in packed_items])

                trucks_used += len(used_bins)
                incomplete = False

            # Print solution for the new bin (if exists)

        else:
            print("No existe solución óptima.")

end_time=time.time()-start_time

print(f"Camiones totales usados: {trucks_used}")
print(f"{end_time} segundos")


**********CLUSTER # 1**********

Número de camiones requeridos: 3

Valor total empaquetado: 41.03

*** CAMIÓN # 1 ***
Capacidad: 27
Clientes empacados: 0, 2, 4, 5, 10, 11, 13, 16, 18, 25, 27, 28, 29, 30, 31, 35, 36, 39, 40, 42
Volumen empaquetado en el camión: 18.89

*** CAMIÓN # 2 ***
Capacidad: 27
Clientes empacados: 50, 54, 56, 57, 60, 61, 64, 68, 69, 71, 75, 81, 83, 84, 87, 88, 90, 91, 92, 93
Volumen empaquetado en el camión: 14.02

*** CAMIÓN # 3 ***
Capacidad: 27
Clientes empacados: 94, 96, 97, 98, 99, 101
Volumen empaquetado en el camión: 8.1204

**********CLUSTER # 2**********

Número de camiones requeridos: 3

Valor total empaquetado: 51.06

*** CAMIÓN # 1 ***
Capacidad: 27
Clientes empacados: 1, 3, 6, 7, 8, 9, 12, 15, 17, 19, 20, 21, 22, 24, 26, 32, 33, 34, 37, 38
Volumen empaquetado en el camión: 14.2392

*** CAMIÓN # 2 ***
Capacidad: 27
Clientes empacados: 41, 43, 44, 45, 46, 47, 48, 49, 51, 52, 53, 55, 58, 59, 62, 63, 65, 66, 67, 70
Volumen empaquetado en el camión: 22.373

In [59]:
my_dict = dict(zip(range(trucks_used), chamions))
mini_tcps = []
for i in my_dict:
    print(i)
    i = list(my_dict[i]) + [0]
    mini_tcps.append(result_array[i,:][:,i])

0
1
2
3
4
5
6


In [60]:
[0]

[0]

## TSP en recursivos (o VRPs)

### Implementacion en los datos (CON LIBRERIA)

In [71]:
from tsp_solver.util import path_cost
import fast_tsp

#28800 segundos son 8 horas

Turno = 0
turnos = [1]
ordenTSP = []

tsp = len(mini_tcps)
i = 0
while True:
    if  Turno <= 28800 and i < len(mini_tcps):
        tour = fast_tsp.find_tour(mini_tcps[i], duration_seconds=0.01)
        print(tour)
        Turno = Turno + path_cost(mini_tcps[i], tour)
        print(Turno)
        i = i+1
    elif i < tsp:
        print(Turno)
        i = i-1
        Turno = 0
        turnos.append(1)
    else:
        break
        

[12, 7, 18, 9, 10, 8, 11, 20, 15, 0, 5, 3, 14, 4, 17, 16, 13, 6, 1, 19, 2]
8100
[0, 10, 18, 4, 12, 9, 17, 16, 19, 6, 3, 1, 14, 7, 11, 5, 13, 15, 20, 8, 2]
18240
[0, 3, 2, 4, 5, 1, 6]
21660
[5, 20, 17, 8, 19, 14, 4, 11, 1, 3, 7, 13, 18, 16, 6, 15, 2, 0, 9, 12, 10]
34680
34680
[19, 9, 12, 10, 5, 1, 11, 4, 18, 13, 3, 7, 16, 6, 15, 0, 2, 14, 8, 17, 20]
10860
[12, 16, 11, 19, 10, 1, 20, 18, 7, 14, 2, 6, 8, 3, 0, 4, 5, 15, 17, 13, 9]
20880
[0, 12, 7, 4, 6, 5, 8, 9, 14, 2, 10, 3, 13, 11, 1, 15]
30720
30720
[0, 12, 7, 4, 6, 5, 8, 9, 14, 2, 10, 3, 13, 11, 1, 15]
9840
[0, 4, 3, 2, 5, 1]
17460


In [72]:
len(turnos)

3